# Category and User Similarity Based Recommender System

### Team Members: 
Shivanshu Arora (126000303), Kritika Kurani (825000784),  Sourabh Shenoy (225009050)

## Inroduction and Problem Statement: 

The web has a large collection of documents. Most of the time spent by a user on the web is often in search of information relevant to his topic of interest. This is where recommender systems come into play. Collaborative Filtering is one of the primary approaches used in recommender systems. However, it suffers from problems such as cold start and a sparse utility matrix. In this project, we implement a hybrid approach where we use collaborative filtering and movie genre which would solve the aforementioned problems, while also attempting to reduce the Root Mean Squared Error (RMSE). We compare this approach with another where we establish movie correlations based on genre compositions and wordnet similarity between genres.
This project attempts to build a simpler model for movie recommendations using minimal and most important features.

## Related Work:

### Previous Work:

A) **_Collaborative Filtering based on User Preferences [1]:_** User Similarity is measured based on Pearson Coefficient, which is measured using the formula given below: <img src="pc.jpg",width=300,height=300>


Where, $\textit{X}$ is a user selected for recommendation, and  $\bar{X}$ is a mean rating of user $\textit{X}$. Then, $σ_X$ is the standard deviation of rating of user $\textit{X}$. $X_i$ is the rating for the ith item by user $\textit{X}$. Let $\textit{Y}$ be the other users. The Pearson correlation coefficient is always between -1 and 1.

B) **_Genre Correlation:_** Each movie belongs to at least one genre. Correlation is found by introducing edges from every preceeding genre to the genres following it. For each edge, the counter for the genre-genre is incrememted. For example, if genre combination is G1 | G2 | G5, then G1 is selected as a criterion genre first and increase by one between a criterion genre G1 and another G2 and G5. Next, G2 is selected as a criterion genre, and increase by one between G2 and G5. After all the values are obtained, the rows and columns are normalised. [1]. User ratings are predcited based on the preferred genres of users, which is obtained explicitly, and the genre correlations of the genres that movie belongs to.

### How our approach differentiates:

A) **_Collaborative Filtering:_**
To measure user similarity, we use Pearson corrleation as above. We also determine the effect of demographics (age and gender) on user similarity and thereby movie rating predictions. 

B) **_Genre Correlation:_**
Previous approaches have user preferred genres explicitly defined, much similar to netflix or movielens which asks new users to provide their preferred genres. Since we do not have that data available, we determine user preferred genres based on the pearson similarity calculated above. The three genres that were found to be prevalent among the neighbors of the user were assigned as that user's preferred genres. 

## Our Approach: 

### Dataset Used:

Movielens 100k dataset has been used in this project. 
1. __u.user__ has information such as user id, age, sex, occupation and zip code.
2. __u.genre__ file contains the list of all genres. 
3. __u.item__ has data about the movie id, movie name, release date, imdb link and a boolean vector representing the combination of genres it belongs to. 
4. __ub.base__ file contains the user id, movie id and the corresponding rating. This data is used for training the recommender.
5. __ub.test__ file contains similar data as ub.base. This file is used for testing the recommender created and evaluation.

We organized users, movies and ratings into separate classes.
The class structure is as follows:
### User Class:
<img src="user.jpg",width=500,height=500>

### Movie Class:
<img src="movie.jpg",width=500,height=500>

### Dataset Info:
<img src="info.jpg",width=500,height=600>


In [1]:
import numpy as np
from nltk.corpus import wordnet as wn

class User:
    def __init__(self,user_id,age,sex,occupation,zipcode):
        self.id = user_id
        self.age = age
        self.sex = sex
        self.occupation = occupation
        self.zipcode = zipcode
        self.avg_rating = 0
        self.pref_genre = []

class Movie:
    def __init__(self,movie_id,name,release_data,imdb_link,genre):
        self.id = movie_id
        self.name = name
        self.release_date = release_data
        self.imdb_link = imdb_link
        self.genre = genre
        self.avg_rating = 0


class Rating:
    def rating_matrix(self,rate_matrix,filename):
        f = open(filename, "r")
        ratings = f.readlines()
        for r in ratings:
            r = r.split("\t")
            rate_matrix[int(r[0])-1][int(r[1])-1] = int(r[2])

The next step was to convert data from files inti useable data structures. We created a class which will read the values from the corresponding files and populate the User, Movie and Rating objects. <br />
This class also has methods that compute genre correlation by genre composition and wordnet similarity of genres.

In [2]:
class Data:
    def __init__(self):
        self.genres_list = self.get_genres()
        self.genre_corr = [[0 for col in range(19)] for row in range(19)]

    def user_data(self):
        users = []
        f = open("./ml-100k/u.user","r")
        lines = f.readlines()
        for line in lines:
            data = line.split("|")
            new_user = User(int(data[0])-1,int(data[1]),data[2],data[3],data[4])
            users.append(new_user)
        return users

    def movies_data(self):
        movies = []
        f = open("./ml-100k/u.item","r")
        lines = f.readlines()
        # making a movie object
        for line in lines:
            new_movie = []
            data = line.split("|")
            genre = {}
            i = 5
            for g in self.genres_list:
                genre[g] = int(data[i])
                i += 1
            new_movie = Movie(int(data[0])-1,data[1],data[2],data[3],genre)
            movies.append(new_movie)
        return movies

    def get_genres(self):
        f_genre = open("./ml-100k/u.genre", "r")
        genres = []
        # getting all the genres
        lines = f_genre.readlines()
        for line in lines:
            line = line.split("|")
            genres.append(line[0])
        return genres[:-1]

    def create_rating_matrix(self,rate_matrix,filename):
        r = Rating()
        r.rating_matrix(rate_matrix,filename)

    def genre_correlation1(self):
        COLUMN_NUM = 19
        data = np.genfromtxt('val.csv', delimiter=',')
        if data.shape[0] % 19 == 0:
            self.genre_corr = data.reshape((-1, 19))
        else:
            data = np.pad(data, (0, COLUMN_NUM - len(data) % COLUMN_NUM), 'constant')
            self.genre_corr = data.reshape((-1, COLUMN_NUM))


    def genre_correlation(self):
        f = open("./ml-100k/u.item", "r")
        lines = f.readlines()
        for i in range(19):
            self.genre_corr[i][i] = 1
        for line in lines:
            data = line.split("|")
            genre = data[5:]
            avg = 0
            k = 0
            genre = [int(x) for x in genre]
            for i in range(19):
                if genre[i] == 1:
                    for j in range(i+1,19):
                        if genre[j] == 1:
                            self.genre_corr[i][j] += 1
                            self.genre_corr[j][i] += 1

        for i in range(19):
            avg = sum(self.genre_corr[i])-1
            if avg != 0:
                self.genre_corr[i] = [(float(x)/avg) for x in self.genre_corr[i]]
                self.genre_corr[i][i] = 1

    def genre_corr_wordnet(self):
        corr = np.zeros((len(self.genres_list), len(self.genres_list)))

        for i in range(len(self.genres_list)):
            g1 = wn.synsets(self.genres_list[i])
            for j in range(len(self.genres_list)):
                g2 = wn.synsets(self.genres_list[j])
                maxval = 0
                for a in g1:
                    x = wn.synset(a._name)
                    for b in g2:
                        y = wn.synset(b._name)
                        res = x.path_similarity(y)
                        maxval = max(maxval,res)

                corr[i][j] = maxval

        return corr

# Predicting the movie ratings

## MoviePredict class

We create a separate class that imports the Data class defined above and retrieves the data stored in files. Next, we work on predicting the rating based on users and movies obtained from the training and testing data files.

## Movie Clustering

Since a movie belongs to one or more genres, the movies need to be clustered so that it now belongs to the most important genre. We used K-means clustering initialized with 19 genres to achieve this. The 19 centres were determined based on certain heuristics which speeds up convergence. This was achieved using sklearn module in Python.

## Ratings Matrix: Users to Genres

The utility matrix in a movie recommender system provides the ratings from a user for a movie. Since we try to establish a relation between movies based on their genres, it is crucial to determine which genres are more interesting to users. To achieve this, we use the ratings available in training data. For each user movie pair, the prevalent genre of the movie is determined from the clustering created above and added to the list of movie genre pair.
After all the ratings of the training data have been analyzed, we have a matrix that stores all the ratings by a user to a particular genre. Next, for every user genre pair, we determine the average of each, so we have the an average user genre rating at the end.

## Average User Ratings

For every user, the average of all the ratings provided by him to the movies is calculated and stored in individual user objects.

## Normalize Ratings

We normalize the ratings i.e. subtract the average user ratings from the user genre ratings in order to account for user personalities. Let's say a user is highly optimistic and provides high ratings (say, 3-5) to all the movies he's watched and another user is critical and provides a rating of 3 to a movie he really liked and a rating of 1 he dislikes. These two users may have similar tastes but if we do not normalize the ratings, these users may seem far from being similar.

## User User Similarity
#### Pearson Similarity:

After we calculate the normalized ratings for each user genre pair, we find the similarity based on these ratings using the Pearson correlation Coefficient defined above. <img src="images\similarity.png",width=350,height=350>

#### Age Bias:
To determine the affect of age on a user user similarity, we peanlize the similarity with the age gap between the users. This bias was calculated as below: <img src="images\agebias.png",width=600,height=600>
Thus the higher the age difference between two users, the more is the similarity penalized. This was based on general observation which shows that people with same age prefer similar movies. This is evident in the results as well, as introducing this bias reduces the overall RMSE obtained on testing data.

#### Sex Bias:
Same gender people might not always like same movies, therefore using only a gender bias in determining the similarity between users increased the overall RMSE. However, when this bias was introduced along with the age bias above, the overall RMSE reduced. This clearly shows that people of same age and same sex tend to like similar movies. <img src="images\sexbias.png",width=400,height=400>


## Bias Form Mean Average - Prediction I

To determine the user rating for a particular movie, we use weighted average of the ratings provided by other similar users which are the top 150 users with highest pearson correlation coefficient to the user in question.
To predict a rating for a movie from a user, we determine the genre of the movie based on the clustering provided above. Then the predicted rating would be the sum of user's average rating for the genre, and of summation of product of similarity and normalized rating for the neighbors divided by the summation of similarities of all the neighbors. <br/> <img src="images\biasform.png",width=350,height=350> <br/>
Where, rating for an active user _a_ and a movie _i_ is $\it{r_{a,i}}$. $\bar{r_a}$ is the average ratings for user _a_ and _P(x,y)_ is the pearson similarity between two users _x_ and _y_. _n_ signifies the nearest neighbors for user _a_. We chose to have the top 150 users with highest similarity as the neighbors for the user under consideration.

## User preferred genres

Since we do not have explicit information about the preferred genres of each user, we determine the majority genres among similar users. 
We determine top 30 users with highest Pearson Correlation with the user in question. For each neighbor and the user himself, we take into account the three genres with highest normalized ratings. The three genres with the highest cumulative score are then designated as the user's preferred genres.

## Computing Genre Correlation

We use two approaches to determine genre correlation:

1). Since a movie belongs to at least one genre, say G1,G2,G5, there definitely exists some relation between the genres. To account for this relation, we create a matrix of genre by genre. Taking G1 as primary genre, we increment the value of G1-G2 by 1 and G1-G5 by 1. Next, considering G2 as primary genre, we increment G2-G5 by 1. After accounting for genre correlations of all the movie in the dataset, we normalize the correlation matrix created.

2). Above approach has a limitation since they utilize genre combinations to establish connections between genres. This is highly dependent on the training set available. We try to alleviate this using WordNet. It provides similarity between words based on inherited hypernym.

Both the approaches provided similar RMSE. If a movie's genres were available in the order of importance instead of aphabetically, the relation between genres could be determined more accurately. This could be crowdsourced and makes for the future scope of the project.

## Genre Correlation Ratings - Prediction II

The first step here is to compute the average movie ratings, which were calculated and assigned to individual movie objects. 
To predict a rating for a user movie pair, we compute summation of correlation values of all pairs of user preferred genre and the genres of the movie under consideration. This value is then multiplied with the average movie rating and divided by the number of user preferred genres, which in our case if fixed to be 3, but in real time scenarios, can vary from user to user. <br/> <img src="images\genre.png",width=250,height=250> <br/>
Where, rating for an active user _a_ and a movie _i_ is $\it{r_{a,i}}$. _up_ refers to the user preferred genres determined above, _mg_ is the genres of the movie _i_. $\it{c_{k,j}}$ is the genre correlation between genres _k_ and _j_. This correlation value can determined by either of the two approaches mentioned above. $\mu_i$ is the average rating for the movie _i_. _|up|_ is the number of preferred genres for user _a_. This value is 3 as mentioned above.

In [3]:
from movie_data import *
from sklearn.cluster import KMeans
import numpy as np

class MoviePredict():
    def __init__(self):
        self.data = Data()
        self.users = []
        self.movies = []
        self.train = []
        self.test = []
        self.genres = []
        self.movie_cluster = None

    def load_data(self):
        self.users = self.data.user_data()
        self.movies = self.data.movies_data()
        self.genres = self.data.get_genres()
        self.train = np.zeros((len(self.users),len(self.movies)))
        self.test = np.zeros((len(self.users), len(self.movies)))
        self.data.create_rating_matrix(self.train,"./ml-100k/ub.base")
        self.data.create_rating_matrix(self.test,"./ml-100k/ub.test")

    def movieClustering(self):
        movie_genre = []
        for m in self.movies:
            mg = []
            for g in self.genres:
                mg.append(m.genre[g])
            movie_genre.append(mg)

        movie_genre = np.array(movie_genre)
        self.movie_cluster = KMeans(n_clusters=len(self.genres)).fit_predict(movie_genre)

    def create_ratingmatrix(self):
        print "Generating Rating matrix"
        avg = np.zeros(len(self.genres))
        ratings = [[ [] for j in range(len(self.genres))] for i in range(len(self.users))]
        for user in self.users:
            for m in self.movies:
                rate = self.train[user.id][m.id]
                if rate != 0:
                    genre = self.movie_cluster[m.id]
                    ratings[user.id][genre].append(rate)

            for i in range(len(self.genres)):
                if ratings[user.id][i] != []:
                    ratings[user.id][i] = np.mean(ratings[user.id][i])
                else:
                    ratings[user.id][i] = 0

        self.ratings = ratings

    def calculate_avgUserRating(self):
        for user in self.users:
            rating = self.ratings[user.id]
            user.avg_rating = np.mean(rating)

    def normalize_rating(self):
        norm = [[[] for j in range(len(self.genres))] for i in range(len(self.users))]
        for user in self.users:
            norm[user.id] = [x - user.avg_rating for x in self.ratings[user.id]]

        self.normRating = norm

    def calculate_pearsonCC(self):
        print "Generating Similarity Matrix"
        maxage = 73
        minage = 7
        pcs = [[0 for j in range(len(self.users))] for i in range(len(self.users))]
        for userA in self.users:
            agerange = max(abs(userA.age - minage), abs(userA.age - maxage))
            for userB in self.users:
                if userA != userB:
                    rateA = [x - userA.avg_rating for x in self.ratings[userA.id]]
                    rateB = [x - userB.avg_rating for x in self.ratings[userB.id]]
                    pcs[userA.id][userB.id] = np.dot(rateA, rateB)
                    # including age bias
                    pcs[userA.id][userB.id] *= float(agerange - abs(userA.age - userB.age)) / agerange
                    # including gender bias
                    # if userA.sex == userB.sex:
                    #     pcs[userA.id][userB.id] *= 1.1
                    # else:
                    #     pcs[userA.id][userB.id] *= 0.9
        self.pcs = pcs

    def guess(self,user,movie,top_n):
        gid = self.movie_cluster[movie]
        pearson = self.pcs[user]
#         agerange = max(abs(self.users[user].age - 7), abs(self.users[user].age - 73))
#         for i in range(len(pearson)):
#             pearson[i] *= float(agerange - abs(self.users[user].age - self.users[i].age)) / agerange
#             if self.users[i].sex == self.users[user].sex:
#                 pearson[i] *= 1.1
#             else:
#                 pearson[i] *= 0.9
        top_similar = np.argsort(pearson)[-top_n:]
        s, c = 0, 0
        for t in top_similar:
            if self.normRating[t][gid] != 0:
                s += self.normRating[t][gid] * pearson[t]
                c += pearson[t]

        rate = self.users[user].avg_rating + float(s) / c
        if rate < 1.0:
            return 1.0
        elif rate > 5.0:
            return 5.0
        else:
            return rate

    def get_rmse(self):
        error = 0.00
        cnt = 0
        err1, err2, c1, c2 = 0, 0, 0, 0

        for user in self.users:
            uid = user.id
            for mov in self.movies:
                mid = mov.id
                if self.test[uid][mid] != 0:
                    pred1 = self.guess(uid, mid, 150)
                    pred2 = self.guess2(uid, mid)
                    pred = 0.85 * pred1 + 0.15 * pred2
                    error += (pred - self.test[uid][mid]) ** 2
                    cnt += 1
                    err1 += (pred1 - self.test[uid][mid]) ** 2
                    err2 += (pred2 - self.test[uid][mid]) ** 2
                    c1 += 1
                    c2 += 1

        print "RMSE=", (float(error) / cnt) ** 0.5
        print "RMSE 1=", (float(err1) / c1) ** 0.5
        print "RMSE 2=", (float(err2) / c2) ** 0.5

    def decide_usergenre(self,top_n=30,genre_no = 3):
        for user in self.users:
            pg = [0]*len(self.genres)
            res = np.argsort(self.normRating[user.id])[-genre_no:]
            for r in res:
                pg[r] += 1
            pearson = self.pcs[user.id]
#             for i in range(len(pearson)):
#                 if self.users[i].sex != self.users[user.id].sex:
#                     pearson[i] *= 0.9
            top_similar = np.argsort(pearson)[-top_n:]
            for t in top_similar:
                res = np.argsort(self.normRating[t])[-genre_no:]
                for r in res:
                    pg[r] += 1

            user.pref_genre = np.argsort(pg)[-genre_no:]
            # user.pref_genre = np.argsort(self.normRating[user.id])[-genre_no:]

    def calculate_movieAvgRating(self):
        for m in self.movies:
            rating = 0
            cnt = 0
            for u in self.users:
                if self.train[u.id][m.id] != 0:
                    rating += self.train[u.id][m.id]
                    cnt += 1
            if cnt == 0:
                m.avg_rating = 0
            else:
                m.avg_rating = float(rating)/cnt

    def guess2(self,user,movie):
        up = self.users[user].pref_genre
        gen = self.movies[movie].genre
        mg = []
        for i in range(len(self.genres)):
            if gen[self.genres[i]] != 0:
                mg.append(i)

        rating = 0
        for k in up:
            for j in mg:
                rating += self.genre_corr[k][j]
        rating *= self.movies[movie].avg_rating
        rate = float(rating)/len(up)
        if rate < 1.0:
            return 1.0
        elif rate > 5.0:
            return 5.0
        else:
            return rate

    def load_allData(self):
        self.load_data()
        self.movieClustering()
        self.create_ratingmatrix()
        self.calculate_avgUserRating()
        self.normalize_rating()
        self.calculate_pearsonCC()
        self.decide_usergenre()
        self.calculate_movieAvgRating()
        self.data.genre_correlation()
        # self.genre_corr = self.data.genre_corr
        self.genre_corr = self.data.genre_corr_wordnet()
        self.get_rmse()

obj = MoviePredict()
obj.load_allData()

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.0, 1, 0.18292682926829268, 0.007317073170731708, 0.02195121951219512, 0.07804878048780488, 0.05609756097560976, 0.0, 0.11219512195121951, 0.00975609756097561, 0.0, 0.03170731707317073, 0.00975609756097561, 0.02195121951219512, 0.06097560975609756, 0.13170731707317074, 0.2048780487804878, 0.05121951219512195, 0.01951219512195122], [0.0, 0.2862595419847328, 1, 0.022900763358778626, 0.16030534351145037, 0.06870229007633588, 0.026717557251908396, 0.0, 0.05343511450381679, 0.03816793893129771, 0.0, 0.011450381679389313, 0.019083969465648856, 0.007633587786259542, 0.05343511450381679, 0.1297709923664122, 0.08015267175572519, 0.03435114503816794, 0.007633587786259542], [0.0, 0.036585365853658534, 0.07317073170731707, 1, 0.4024390243902439, 0.12195121951219512, 0.0, 0.0, 0.0, 0.024390243902439025, 0.0, 0.012195121951219513, 0.2073170731707317, 0.0, 0.024390243902439025, 0.04878048780487805, 0.036585365853658534, 0.01219512195121951

## Evaluation
### Test Data
We used the movielens dataset i.e. ml-100k. Based on the ratings provided by the users during training, we predicted the ratings for other user movie pairs in testing data set. After all the ratings were predicted, RMSE was calculated for various demographic combinations and the results were plotted.

### Results and Evaluations

<img src="images\image.png",width=500,height=600>
From the above graph we can see that as we start to bias our model with age and sex, our RMSE starts dropping. With age and sex bias, our RMSE comes out to be around 1.37 as compared to 1.40 without any bias. This can be due to the fact that users of similar ages and same sex tend to watch the similar movies and give almost the same ratings. However, the drop in the RMSE is not significant enough to generalize the demographic biases.

<img src="images\image1.png",width=500,height=600>
In the above case, we are modelling our recommender system with genre-genre correlation. The RMSE drop by using age and sex biases as before. However, the RMSE values from genre-genre correlation is higher than that of the bias form mean average because in this case, because we are implicitly finding out the user's preferred genre which might not be the best representation.

<img src="images\image2.png",width=500,height=600>
The weights used in case of no bias were 0.85 for bias form mean average and 0.15 for genre correlation predictions. These weights reduced the RMSE for the test data where no demographic biases were used for determining similarity . However, this division or choice of weights might not be appropriate when demographic bias i.e. age and sex biases were introduced to determine user user similarity. To reduce RMSE for the demographic bias cases, we should choose different weights appropriate for the scenario.

## Conclusion and next steps

### Things we learned:

1). Design a hybrid recommender system, that builds user profile based on the genres of the movies watched by him and determines the rating of a movie by user based on movie genre and user user similarity.
<br/>2). Features of the items and users that can be used to determine ratings and user-user similarity. For movie, we used its genre, while for user we build simialrity based on the movie genres it has watched, the sex of the user and his/her age.
<br/>3). Demographic similarity between users cannot be generalized.

### Learned concepts used in the project:
1). Recommender system designs: Model based, content based and hybrid models.
<br/>2). User-user Collaborative filtering.
<br/>3). RMSE evaluations

### Next steps:
1). The major reason for a high RMSE for genre correlation based prediction was that the user's preferred genre was implicitly calculated. We took into account the genres that were most watched by the user and its other similar users. This might not provide correct estimations for preferred genres. Taking explicit input from user will definitely provide more accurate predictions.
<br/>2). An efficient method for prediction would be to find latent factors and reduce the rating utility matrix into a lower dimension by mapping users and movies into this latent factor space. The techniques used in the project can then be used with these new features and the ratings can be linearlly blended to provide better ratings.

## References:

[1]. [Kyung-Rog Kim, Ju-Ho Lee and Jae-Hee Byeon, "Recommender System Using the Movie Genre Similarity in Mobile Service"](http://ieeexplore.ieee.org/document/5575081/) <br/>
[2]. [Sang-Min Choi, Da-Jung Cho, Yo-sub Han, "Recommender Systems Using Category Correlations Based on WordNet Similarity"](http://ieeexplore.ieee.org/document/7079614/authors)